In [1]:
import pandas as pd
import numpy as np


chrom = 22
df = pd.read_pickle('chrom_%d_real.pkl' % chrom)
df.set_index(['gene', 'intron'], inplace=True)
df.sort_index(inplace=True)
df.head()

chrom  lmm-pval  lmm-rank-pval       pos  qep-pval  \
gene            intron                                                       
ENSG00000015475 6          22  0.006159       0.010802  18183494  0.017806   
                6          22  0.850907       0.856190  18183623  0.930428   
                6          22  0.149487       0.285375  18183724  0.172327   
                6          22  0.850907       0.856190  18184011  0.930428   
                6          22  0.130812       0.133490  18184169  0.117560   

                                 snp_id  
gene            intron                   
ENSG00000015475 6       snp_22_18183494  
                6       snp_22_18183623  
                6       snp_22_18183724  
                6       snp_22_18184011  
                6       snp_22_18184169

# Bonferroni correction

In [2]:
ntests = len(df.index.unique())
print("Number of tests: %d" % ntests)

# Bonferroni within gene-intron
def pval_agg(x):
    return np.clip(np.min(x) * len(x), 0, 1)

def method_agg(x):
    return x[0]
    
def gene_intron_wise(df):
    df = df.groupby(level=[0, 1]).agg({
        'lmm-pval': pval_agg,
        'lmm-rank-pval': pval_agg,
        'qep-pval': pval_agg
    })
    return df

df = gene_intron_wise(df)

Number of tests: 1740


In [3]:
df.head()

qep-pval  lmm-pval  lmm-rank-pval
gene            intron                                   
ENSG00000015475 6       1.000000  1.000000       1.000000
                7       0.874542  0.004949       0.095534
ENSG00000025770 1       1.000000  0.000683       1.000000
                2       1.000000  0.000004       1.000000
                3       0.148910  0.239379       0.313101

In [4]:
data = pd.DataFrame(columns=('pval', 'method'))

def append_df(data, df, name):
    pval = df['%s-pval' % name]
    data = pd.concat([data, pd.DataFrame({'pval': pval, 'method': name})])
    return data

data = append_df(data, df, 'lmm')
data = append_df(data, df, 'lmm-rank')
data = append_df(data, df, 'qep')

In [5]:
data.head()

,method,pval
"(ENSG00000015475, 6)",lmm,1.000000
"(ENSG00000015475, 7)",lmm,0.004949
"(ENSG00000025770, 1)",lmm,0.000683
"(ENSG00000025770, 2)",lmm,0.000004
"(ENSG00000025770, 3)",lmm,0.239379


In [6]:
from bokeh.io import push_notebook, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure, show, output_file
from bokeh.charts import BoxPlot, Bar
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [7]:
def plot_curve(data, perc=False, exclude=False, lthr=-5, rthr=-0.30103):
    
    nhits = {'lmm-rank':[], 'qep':[]}
    if not exclude:
        nhits['lmm'] = []
    
    pts = np.logspace(lthr, rthr)
    for pt in pts:

        if perc:
            grouped = data.groupby(data['method']).agg({'method': lambda x: x[0], 'pval': lambda x: 100*sum(x*len(x)<=pt)/float(len(x))})
        else:
            grouped = data.groupby(data['method']).agg({'method': lambda x: x[0], 'pval': lambda x: sum(x*len(x)<=pt)})


        nhits['lmm-rank'].append(grouped.loc['lmm-rank', 'pval'])
        if not exclude:
            nhits['lmm'].append(grouped.loc['lmm', 'pval'])
        nhits['qep'].append(grouped.loc['qep', 'pval'])

    p = figure(title="Power", tools=['save,reset'])
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = 'P-value threshold'
    if perc:
        p.yaxis.axis_label = 'Percentage of hits'
    else:
        p.yaxis.axis_label = 'Number of hits'
    p.line(pts, nhits['lmm-rank'], color='green', legend='lmm-rank')
    if not exclude:
        p.line(pts, nhits['lmm'], color='blue', legend='lmm')
    p.line(pts, nhits['qep'], color='red', legend='qep')
    p.legend.location = "bottom_right"
    return p

In [8]:
p1 = plot_curve(data, True)
p2 = plot_curve(data, True, True)
p3 = plot_curve(data, False)
p4 = plot_curve(data, False, True)
show(gridplot([[p1,p2],[p3,p4]], plot_width=400, plot_height=400))

In [9]:
lthr = -5
rthr = -1
p1 = plot_curve(data, True, lthr=lthr, rthr=rthr)
p2 = plot_curve(data, True, True, lthr=lthr, rthr=rthr)
p3 = plot_curve(data, False, lthr=lthr, rthr=rthr)
p4 = plot_curve(data, False, True, lthr=lthr, rthr=rthr)
show(gridplot([[p1,p2],[p3,p4]], plot_width=400, plot_height=400))

In [10]:
lthr = -10
rthr = -2
p1 = plot_curve(data, True, lthr=lthr, rthr=rthr)
p2 = plot_curve(data, True, True, lthr=lthr, rthr=rthr)
p3 = plot_curve(data, False, lthr=lthr, rthr=rthr)
p4 = plot_curve(data, False, True, lthr=lthr, rthr=rthr)
show(gridplot([[p1,p2],[p3,p4]], plot_width=400, plot_height=400))